In [231]:
import os
import sys
from os.path import join as pj
from copy import deepcopy

In [677]:
class GalfitComponent:
    def __init__(self, 
                 component_name = "", 
                 component_number = 0, 
                 param_prefix = " "
                ):
        
        self.component_name = component_name
        self.component_number = component_number
        self.param_prefix = param_prefix
        self.param_numbers = [0]
        key = "Component type"
        self.param_desc = {key : key}
        # tuple in value for printing
        self.param_values = {key : (component_name, "")}
        self.param_fix = {key : ""}
        
    def check_lengths(self):
        
        if not all(len(ec) == len(self.param_numbers) for ec in (self.param_numbers, self.param_values, self.param_desc, self.param_fix)):
            print("Length of array containing parameter numbers, values, and descriptions must be the same!")
            
            print("param_numbers:", end = " ")
            print(*self.param_numbers, sep = "\n")
            
            print("param_values:", end = " ")
            print(*self.param_values.keys(), sep = "\n")
            
            print("param_fix:", end = " ")
            print(*self.param_fix.keys(), sep = "\n")
            
            print("param_desc:", end = " ")
            print(*self.param_desc.keys(), sep = "\n")
            
            raise(AssertionError())
        
    def __str__(self):
        output_str = ""
        
        if self.component_name not in ("power", "fourier"):
            output_str = f"# Component number: {self.component_number}\n"
        
        self.check_lengths()
        
        for num, val, fix, desc in zip(self.param_numbers, self.param_values.values(), self.param_fix.values(), self.param_desc.values()):
            # Skip component type
            if isinstance(val, (int, float)):
                if num == "Z":
                    line = f"{self.param_prefix}{num}) {val:<11} {fix}"
                else:
                    line = f"{self.param_prefix}{num}) {val:<11.4f} {fix}"
                
                if len(f"{self.param_prefix}{num}") >= 3: line = line.lstrip()
                if len(line.split()[0]) >= 4: line = line[:4] + line[5:]
                
                line = f"{line:<23} # {desc}\n"
                
            else:
                # position #, fourier, etc.
                if isinstance(val[0], (int, float)):
                    line = f"{self.param_prefix}{num}) {val[0]:<7.4f} {val[1]:.4f} {fix}" # {desc}\n"
                # comp name
                else:
                    line = f"{self.param_prefix}{num}) {val[0]}" # {desc}\n"
                    
                line = f"{line:<23} # {desc}\n"
                
            output_str += line

        return output_str

    def to_file(self, filename, *args):
        with open(filename, "w+") as f:
            f.write(str(self))
            f.write("\n")
            
            # *args for writing in additional classes at the same time (save I/O)
            comp_names = [c.component_name for c in args]
            with_fourier = "fourier" in comp_names
            if with_fourier:
                fourier_index = comp_names.index("fourier")
            
            for i, component in enumerate(args):
                f.write(str(component))
                if i != fourier_index - 1:
                    f.write("\n")
            
    def add_skip(self, skip_val = 0):
        key = "skip"
        self.param_numbers.append("Z")
        self.param_values[key] = skip_val
        self.param_desc[key] = "Skip this model in output image?  (yes=1, no=0)"
        self.param_fix[key] = ""
        
    def update_param_values(self):
        # Values are the only thing that will change via instance.param_name
        # We need to update the dictionary that holds all of them since I use
        # a deepcopy to save myself some lines of code/typing (cheeky)
        self.param_values.update({k:v for k,v in self.__dict__.items() if k in self.param_values})
        
        # Either 
        # instance.param_name = #
        # instance.update_param_values()
        # or
        # instance.param_values["param_name"] = #
        # works
            
    # #https://stackoverflow.com/a/35282351
    # def __iter__(self):
    #     return self.__dict__.iteritems()

In [678]:
class Sersic(GalfitComponent):
    def __init__(self, component_number, **kwargs):
        
        self.position = kwargs.get("position", (0.0000, 0.0000))
        self.magnitude = kwargs.get("magnitude", 13)
        self.effective_radius = kwargs.get("effective_radius", 10)
        self.sersic_index = kwargs.get("sersic_index", 4)
        self.axis_ratio = kwargs.get("axis_ratio", 0.6)
        self.position_angle = kwargs.get("position_angle", 0)
        #self.skip = kwargs.get("skip", 0)
        
        param_dict = deepcopy(self.__dict__)
        
        GalfitComponent.__init__(self, component_name = "sersic", component_number = component_number)
        
        self.param_numbers += [1,3,4,5,9,10]
        
        self.param_values.update(param_dict)
                
        self.param_fix.update({k : 1 for k in param_dict.keys()})
        self.param_fix["position"] = "0 0"
        
        pd = self.param_desc
        pd["position"] = "Position x, y"
        pd["magnitude"] = "Integrated magnitude"
        pd["effective_radius"] = "R_e (effective radius)   [pix]"
        pd["sersic_index"] = "Sersic index n (de Vaucouleurs n=4)"
        pd["axis_ratio"] = "Axis ratio (b/a)"
        pd["position_angle"] = "Position angle (PA) [deg: Up=0, Left=90]"

In [679]:
class Power(GalfitComponent):
    def __init__(self, **kwargs):
        
        #self.component_type = "power"
        self.inner_rad = kwargs.get("inner_rad", 0)
        self.outer_rad = kwargs.get("outer_rad", 20)
        self.cumul_rot = kwargs.get("cumul_rot", 90)
        self.powerlaw = kwargs.get("powerlaw", 1)
        self.inclination = kwargs.get("inclination", 15)
        self.sky_position_angle = kwargs.get("sky_position_angle", 45)
        #self.skip = kwargs.get("skip", 0)
        
        param_dict = deepcopy(self.__dict__)
        
        GalfitComponent.__init__(self, component_name = "power", param_prefix = "R")
        
        self.param_numbers += [1,2,3,4,9,10]
        
        self.param_values.update(param_dict)
        
        self.param_fix.update({k : 1 for k in param_dict.keys()})
        self.param_fix["inner_rad"] = 0
        self.param_fix["outer_rad"] = 0
        
        pd = self.param_desc
        pd["Component type"] = "PA rotation func. (power, log, none)"
        pd["inner_rad"] = "Spiral inner radius [pixels]"
        pd["outer_rad"] = "Spiral outer radius [pixels]"
        pd["cumul_rot"] = "Cumul. rotation out to outer radius [degrees]"
        pd["powerlaw"] = "Asymptotic spiral powerlaw"
        pd["inclination"] = "Inclination to L.o.S. [degrees]"
        pd["sky_position_angle"] = "Sky position angle"
        
#     def add_fn(self, n = {1 : (0.05, 45), 3 : (0.05, 25)}):

#         for num, values in n.items():
#             key = f"F{num}"
#             self.param_numbers.append(num)
#             self.param_values[key] = values
#             self.param_desc[key] = f"Azim. Fourier mode {num}, amplitude, & phase angle"
#             self.param_fix[key] = "1 1"

In [680]:
class Fourier(GalfitComponent):
    def __init__(self, n = {1 : (0.05, 45), 3 : (0.05, 25)}):
        GalfitComponent.__init__(self, component_name = "fourier", param_prefix = "F")      
        # normal rules don't apply here
        self.param_numbers = []
        self.param_values = {}
        self.param_desc = {}
        self.param_fix = {}
        
        def include_fn(self, n:dict):

            for num, values in n.items():
                key = f"{self.param_prefix}{num}"
                self.param_numbers.append(num)
                self.param_values[key] = values
                self.param_desc[key] = f"Azim. Fourier mode {num}, amplitude, & phase angle"
                self.param_fix[key] = "1 1"
        
        include_fn(self, n = n)

In [681]:
class Sky(GalfitComponent):
    def __init__(self, component_number, **kwargs):
        #self.component_type = "power"
        self.sky_background = kwargs.get("sky_background", 1000)
        self.dsky_dx = kwargs.get("dsky_dx", 0)
        self.dsky_dy = kwargs.get("dsky_dy", 0)
        
        param_dict = deepcopy(self.__dict__)
        
        GalfitComponent.__init__(self, component_name = "sky", component_number = component_number)
        
        self.param_numbers += [1,2,3]
        
        self.param_values.update(param_dict)
        
        self.param_fix.update({k : 1 for k in param_dict.keys()})
        
        pd = self.param_desc
        pd["sky_background"] = "Sky background at center of fitting region [ADUs]"
        pd["dsky_dx"] = "dsky/dx (sky gradient in x)     [ADUs/pix]"
        pd["dsky_dy"] = "dsky/dy (sky gradient in y)     [ADUs/pix]"

In [682]:
bulge = Sersic(1, position = (25,25))
disk  = Sersic(2, position = (25,25))
arms  = Power()
#arms.add_fn()
fourier = Fourier()
sky   = Sky(3)

In [683]:
print(bulge)
print(disk)
print(arms)
print(fourier)
print(sky)

# Component number: 1
 0) sersic              # Component type
 1) 25.0000 25.0000 0 0 # Position x, y
 3) 13.0000     1       # Integrated magnitude
 4) 10.0000     1       # R_e (effective radius)   [pix]
 5) 4.0000      1       # Sersic index n (de Vaucouleurs n=4)
 9) 0.6000      1       # Axis ratio (b/a)
10) 0.0000      1       # Position angle (PA) [deg: Up=0, Left=90]

# Component number: 2
 0) sersic              # Component type
 1) 25.0000 25.0000 0 0 # Position x, y
 3) 13.0000     1       # Integrated magnitude
 4) 10.0000     1       # R_e (effective radius)   [pix]
 5) 4.0000      1       # Sersic index n (de Vaucouleurs n=4)
 9) 0.6000      1       # Axis ratio (b/a)
10) 0.0000      1       # Position angle (PA) [deg: Up=0, Left=90]

R0) power               # PA rotation func. (power, log, none)
R1) 0.0000      0       # Spiral inner radius [pixels]
R2) 20.0000     0       # Spiral outer radius [pixels]
R3) 90.0000     1       # Cumul. rotation out to outer radius [degr

In [684]:
bulge.to_file("tester", disk, arms, fourier, sky)